<a href="https://colab.research.google.com/gist/PaulGG-Code/88ef96c25b893f25387c1dc9b5343048/cra-req.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install yfiles package and complete imports
%pip install yfiles_jupyter_graphs pandas --quiet
from yfiles_jupyter_graphs import GraphWidget
import pandas as pd
from collections import defaultdict
from typing import Dict

# Import and enable Google Colab custom widget manager (required for running in Colab)
try:
    import google.colab
    from google.colab import output
    output.enable_custom_widget_manager()
except:
    pass

# Load CSV files
csv_files = {
    "Resellers": "/content/CRA-Requirements - Resellers_ other economic actors.csv",
    "Distributors": "/content/CRA-Requirements - Distributors.csv",
    "Importers": "/content/CRA-Requirements - Importers.csv",
    "Important_Critical_Software": "/content/CRA-Requirements - Important_Critical_Software.csv",
    "IMPORTANT_Critical_products": "/content/CRA-Requirements - IMPORTANT_Critical_products.csv",
    "non_IMPORTANT_products_Software": "/content/CRA-Requirements - non-IMPORTANT products_Software.csv",
    "Developers_Open_Source": "/content/CRA-Requirements - Developers of open source software.csv"
}

# Extract nodes and common requirements
nodes = []
common_requirements = defaultdict(list)

for prefix, file_path in csv_files.items():
    df = pd.read_csv(file_path)
    for idx, row in df.iterrows():
        requirement = row.get('Requirement') or row.get('Unnamed: 1')
        reference = row.get('Reference') or row.get('Unnamed: 2')
        if pd.notna(requirement) and pd.notna(reference) and "Requirement" not in requirement:
            node_id = f"{prefix}-{idx}"
            nodes.append({
                "id": node_id,
                "properties": {"label": requirement.strip(), "Article": reference.strip(), "ActorType": prefix}
            })
            normalized_req = requirement.strip().lower()
            common_requirements[normalized_req].append(node_id)

# Create edges connecting nodes with common requirements
edges = []
for req, nodes_list in common_requirements.items():
    if len(nodes_list) > 1:
        for i in range(len(nodes_list)-1):
            edges.append({
                "id": f"{nodes_list[i]}_{nodes_list[i+1]}",
                "start": nodes_list[i],
                "end": nodes_list[i+1],
                "properties": {"label": "COMMON_REQUIREMENT"}
            })

# Define the graph widget
w = GraphWidget()
w.nodes = nodes
w.edges = edges
w.directed = True

# Node color mapping
def custom_node_color_mapping(node: Dict):
    colors = {
        "Resellers": "#e8a876",
        "Distributors": "#ede772",
        "Importers": "#93e876",
        "Important_Critical_Software": "#76c0e8",
        "IMPORTANT_Critical_products": "#e362d4",
        "non_IMPORTANT_products_Software": "#a876e8",
        "Developers_Open_Source": "#76e8e3"
    }
    return colors.get(node['properties']['ActorType'], "#CCCCCC")

# Define the graph
w = GraphWidget()
w.nodes = nodes
w.edges = edges
w.directed = True
w.set_node_color_mapping(custom_node_color_mapping)

# Display the main graph
display(w)


GraphWidget(layout=Layout(height='800px', width='100%'))

This subgraph focuses on the nodes and edges related to Article 16 of the AI regulation. It provides a visual representation of the specific requirements and their relationships within Article 16, allowing for a more targeted analysis of this particular section of the regulation.

In [ ]:
article = "article 16"
article_nodes = [node for node in w.nodes if article in node['properties']['Article'].lower()]
article_edges = [edge for edge in w.edges if edge['start'] in [n['id'] for n in article_nodes] or edge['end'] in [n['id'] for n in article_nodes]]

article_graph = GraphWidget()
article_graph.nodes = article_nodes
article_graph.edges = article_edges
article_graph.directed = True
article_graph.set_node_color_mapping(custom_node_color_mapping)
display(article_graph)


GraphWidget(layout=Layout(height='500px', width='100%'))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')